In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import nibabel as nib
import os
import time

import pandas as pd
import numpy as np
import cv2
from skimage.transform import resize

from mricode.utils import log_textfile, createPath, data_generator
from mricode.utils import copy_colab
from mricode.utils import return_iter
from mricode.utils import return_csv
from mricode.config import config

from mricode.models.SimpleCNN_small import SimpleCNN
from mricode.models.DenseNet_NoDict import MyDenseNet

import tensorflow as tf
from tensorflow.keras.layers import Conv3D
from tensorflow import nn
from tensorflow.python.ops import nn_ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.keras.engine.base_layer import InputSpec
from tensorflow.python.keras.utils import conv_utils

tf.__version__

'2.0.0'

In [3]:
tf.test.is_gpu_available()

True

In [4]:
path_output = './output/'
path_tfrecords = '/data2/res64/down/'
path_csv = '/data2/csv/'
filename_res = {'train': 'intell_residual_train.csv', 'val': 'intell_residual_valid.csv', 'test': 'intell_residual_test.csv'}
filename_final = filename_res
sample_size = 'site16_allimages'
batch_size = 8
onlyt1 = False
modelname = 'densenet_allimages64'
Model = SimpleCNN
#Model = MyDenseNet
t1_mean=1.3779395849814497
t1_std=3.4895845243139503
t2_mean=2.22435586968901
t2_std=5.07708743178319
ad_mean=1.3008901218593748e-05
ad_std=0.009966655860940228
fa_mean=0.0037552628409334037
fa_std=0.012922319568740915
md_mean=9.827903909139596e-06
md_std=0.009956973204022659
rd_mean=8.237404999587111e-06
rd_std=0.009954672598675338

In [5]:
train_df, val_df, test_df, norm_dict = return_csv(path_csv, filename_final, False)

In [17]:
versionkey = 'cropped128'
train_iter = config[versionkey]['iter_train']
val_iter = config[versionkey]['iter_val']
test_iter = config[versionkey]['iter_test']
t1_mean = config[versionkey]['norm']['t1'][0]
t1_std= config[versionkey]['norm']['t1'][1]
t2_mean=config[versionkey]['norm']['t2'][0]
t2_std=config[versionkey]['norm']['t2'][1]
ad_mean=config[versionkey]['norm']['ad'][0]
ad_std=config[versionkey]['norm']['ad'][1]
fa_mean=config[versionkey]['norm']['fa'][0]
fa_std=config[versionkey]['norm']['fa'][1]
md_mean=config[versionkey]['norm']['md'][0]
md_std=config[versionkey]['norm']['md'][1]
rd_mean=config[versionkey]['norm']['rd'][0]
rd_std=config[versionkey]['norm']['rd'][1]

In [18]:
batch = next(iter(train_iter))

In [19]:
t1 = (tf.cast(batch['t1'], tf.float32)-t1_mean)/t1_std
t2 = (batch['t2']-t2_mean)/t2_std
print(np.mean(t1), np.std(t1), np.mean(t2), np.std(t2))
if True:
    ad = batch['ad']
    ad = tf.where(tf.math.is_nan(ad), tf.zeros_like(ad), ad)
    ad = (ad-ad_mean)/ad_std
    fa = batch['fa']
    fa = tf.where(tf.math.is_nan(fa), tf.zeros_like(fa), fa)
    fa = (fa-fa_mean)/fa_std
    md = batch['md']
    md = tf.where(tf.math.is_nan(md), tf.zeros_like(md), md)
    md = (md-md_mean)/md_std
    rd = batch['rd']
    rd = tf.where(tf.math.is_nan(rd), tf.zeros_like(rd), rd)
    rd = (rd-rd_mean)/rd_std
    print(np.mean(ad), np.std(ad), np.mean(fa), np.std(fa))
    print(np.mean(md), np.std(md), np.mean(rd), np.std(rd))

0.0058205845 1.0090235 -0.08604767 0.9554573


KeyError: 'ad'

In [16]:
if True:
  t1_mean = 0.
  t1_std = 0.
  t2_mean = 0.
  t2_std = 0.
  n = 0.
  for b in train_iter:
      log_textfile('meanstcr256_new.log',str(n))
      t1_mean += np.mean(b['t1'])
      t1_std += np.std(b['t1'])
      t2 = b['t2']
      t2_mean += np.mean(t2)
      t2_std += np.std(t2)
      n += 1
      log_textfile('meanstcr256_new.log',str((t1_mean/n, t1_std/n, t2_mean/n, t2_std/n)))

  t1_mean /= n
  t1_std /= n
  t2_mean /= n
  t2_std /= n

0.0


/usr/local/lib/python3.5/dist-packages/numpy/core/_methods.py:193: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


(9.81284585595131, 30.288950321353788, nan, nan)
1.0
(10.43483892083168, 29.85030963785138, nan, nan)
2.0
(10.402180661757788, 29.125928471280588, nan, nan)
3.0
(10.879297412931919, 28.874293025921894, nan, nan)
4.0
(10.925156563520432, 29.01586060587732, nan, nan)
5.0


KeyboardInterrupt: 

In [10]:
t2 = np.asarray(b['t2'], dtype=np.float64)


In [ ]:
tf.math.reduce_mean(b['t1'])

In [8]:
a = np.asarray(b['t2'])
a = a.copy()
a[np.isnan(a)] = 0   

In [11]:
np.std(a)

nan

In [31]:
np.

0

In [9]:
decoder = np.vectorize(lambda x: x.decode('UTF-8'))
#decoder(batch['subjectid'])

In [25]:
b['t2']

TensorShape([8, 256, 256, 256, 1])

In [10]:
error

ResourceExhaustedError: OOM when allocating tensor with shape[7,128,128,128,1] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Cast] name: Cast/

In [9]:
log_textfile('meanstd256.log',str((t1_mean, t1_std, t2_mean, t2_std)))

(1.6431085066123967, 3.774774262557981, 2.3879970019972974, 5.233771497359756)


In [15]:
train_iter = config['cropped64']['iter_train']
val_iter = config['cropped64']['iter_val']
test_iter = config['cropped64']['iter_test']
if True:
  t1_mean = 0.
  t1_std = 0.
  t2_mean = 0.
  t2_std = 0.
  ad_mean = 0.
  ad_std = 0.
  fa_mean = 0.
  fa_std = 0.
  md_mean = 0.
  md_std = 0.
  rd_mean = 0.
  rd_std = 0.
  n = 0.
  for b in train_iter:
      print(n)
      t1_mean += np.mean(b['t1'])
      t1_std += np.std(b['t1'])
      t2_mean += np.mean(b['t2'])
      t2_std += np.std(b['t2'])
      a = np.asarray(b['ad'])
      a = a.copy()
      a[np.isnan(a)] = 0
      ad_mean += np.mean(a)
      ad_std += np.std(a)
      a = np.asarray(b['fa'])
      a = a.copy()
      a[np.isnan(a)] = 0
      fa_mean += np.mean(a)
      fa_std += np.std(a)
      a = np.asarray(b['md'])
      a = a.copy()
      a[np.isnan(a)] = 0
      md_mean += np.mean(a)
      md_std += np.std(a)
      a = np.asarray(b['rd'])
      a = a.copy()
      a[np.isnan(a)] = 0
      rd_mean += np.mean(a)
      rd_std += np.std(a)
      n += 1
  t1_mean /= n
  t1_std /= n
  t2_mean /= n
  t2_std /= n
  ad_mean /= n
  ad_std /= n
  fa_mean /= n
  fa_std /= n
  md_mean /= n
  md_std /= n
  rd_mean /= n
  rd_std /= n

0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
184.0


In [16]:
t1_mean, t1_std, t2_mean, t2_std, ad_mean, ad_std, fa_mean, fa_std, md_mean, md_std, rd_mean, rd_std

(0.11312570418978232,
 0.14135518265387828,
 27.98422794577516,
 45.90435212793337,
 0.0007140914352312044,
 0.1471820359293305,
 0.18089792297946083,
 0.18397762627431558,
 0.0005650996822530141,
 0.14559568945663873,
 0.0004971999132421456,
 0.13345626959299658)

In [ ]:
all_files = glob.glob('/data/home/benedikt_d_schifferer/data_T2_lowerres_cropped128/' + '*.gz') 
all_files += glob.glob('/data/home/benedikt_d_schifferer/data_T1_lowerres_cropped128/' + '*.gz')
all_files = glob.glob('/data/home/benedikt_d_schifferer/data_T1_T2_201909/' + '*.gz') 

train_iter = data_generator(list(train_df['subjectkey'].values), all_files, batch_size, 
                            input_shape=(256,256,256))
val_iter = data_generator(list(val_df['subjectkey'].values), all_files, batch_size, 
                            is_validation_data=True, 
                            input_shape=(256,256,256))
test_iter = data_generator(list(test_df['subjectkey'].values), all_files, batch_size, 
                            is_validation_data=True, 
                            input_shape=(256,256,256))

In [ ]:
if True:
  t1_mean = 0.
  t1_std = 0.
  t2_mean = 0.
  t2_std = 0.
  n = 0.
  for b in train_iter:
      t1_mean += np.mean(b['t1'])
      t1_std += np.std(b['t1'])
      t2_mean += np.mean(b['t2'])
      t2_std += np.std(b['t2'])
      n += np.asarray(b['t1']).shape[0]
      print(n)

  t1_mean /= n
  t1_std /= n
  t2_mean /= n
  t2_std /= n

In [ ]:
error

In [ ]:
norm_dict

In [ ]:
cat_cols = {'female': 2, 'race.ethnicity': 5, 'high.educ_group': 4, 'income_group': 8, 'married': 6}
num_cols = [x for x in list(val_df.columns) if '_norm' in x]

In [ ]:
def calc_loss_acc(out_loss, out_acc, y_true, y_pred, cat_cols, num_cols, norm_dict):
  for col in num_cols:
    tmp_col = col
    tmp_std = norm_dict[tmp_col.replace('_norm','')]['std']
    tmp_y_true = tf.cast(y_true[col], tf.float32).numpy()
    tmp_y_pred = np.squeeze(y_pred[col].numpy())
    if not(tmp_col in out_loss):
      out_loss[tmp_col] = np.sum(np.square(tmp_y_true-tmp_y_pred))
    else:
      out_loss[tmp_col] += np.sum(np.square(tmp_y_true-tmp_y_pred))
    if not(tmp_col in out_acc):
      out_acc[tmp_col] = np.sum(np.square((tmp_y_true-tmp_y_pred)*tmp_std))
    else:
      out_acc[tmp_col] += np.sum(np.square((tmp_y_true-tmp_y_pred)*tmp_std))
  for col in list(cat_cols.keys()):
    tmp_col = col
    if not(tmp_col in out_loss):
      out_loss[tmp_col] = tf.keras.losses.SparseCategoricalCrossentropy()(tf.squeeze(y_true[col]), tf.squeeze(y_pred[col])).numpy()
    else:
      out_loss[tmp_col] += tf.keras.losses.SparseCategoricalCrossentropy()(tf.squeeze(y_true[col]), tf.squeeze(y_pred[col])).numpy()
    if not(tmp_col in out_acc):
      out_acc[tmp_col] = tf.reduce_sum(tf.dtypes.cast((y_true[col] == tf.argmax(y_pred[col], axis=-1)), tf.float32)).numpy()
    else:
      out_acc[tmp_col] += tf.reduce_sum(tf.dtypes.cast((y_true[col] == tf.argmax(y_pred[col], axis=-1)), tf.float32)).numpy()    
  return(out_loss, out_acc)

def format_output(out_loss, out_acc, n, cols, print_bl=False):
  loss = 0
  acc = 0
  output = []
  for col in cols:
    output.append([col, out_loss[col]/n, out_acc[col]/n])
    loss += out_loss[col]/n
    acc += out_acc[col]/n
  df = pd.DataFrame(output)
  df.columns = ['name', 'loss', 'acc']
  if print_bl:
    print(df)
  return(loss, acc, df)

@tf.function
def train_step(X, y, model, optimizer, cat_cols, num_cols):
  with tf.GradientTape() as tape:
    predictions = model(X)
    i = 0
    loss = tf.keras.losses.MSE(tf.cast(y[num_cols[i]], tf.float32), tf.squeeze(predictions[num_cols[i]]))
    for i in range(1,len(num_cols)):
      loss += tf.keras.losses.MSE(tf.cast(y[num_cols[i]], tf.float32), tf.squeeze(predictions[num_cols[i]]))
    for col in list(cat_cols.keys()):
      loss += tf.keras.losses.SparseCategoricalCrossentropy()(tf.squeeze(y[col]), tf.squeeze(predictions[col]))
  gradients = tape.gradient(loss, model.trainable_variables)
  mean_std = [x.name for x in model.non_trainable_variables if ('batch_norm') in x.name and ('mean' in x.name or 'variance' in x.name)]
  with tf.control_dependencies(mean_std):
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return(y, predictions, loss)

@tf.function
def test_step(X, y, model):
  predictions = model(X)
  return(y, predictions)

def epoch(data_iter, df, model, optimizer, cat_cols, num_cols, norm_dict):
  out_loss = {}
  out_acc = {}
  n = 0.
  n_batch = 0.
  total_time_dataload = 0.
  total_time_model = 0.
  start_time = time.time()
  for batch in data_iter:
    total_time_dataload += time.time() - start_time
    start_time = time.time()
    t1 = (tf.cast(batch['t1'], tf.float32)-t1_mean)/t1_std
    t2 = (batch['t2']-t2_mean)/t2_std
    if False:
        ad = batch['ad']
        ad = tf.where(tf.math.is_nan(ad), tf.zeros_like(ad), ad)
        ad = (ad-ad_mean)/ad_std
        fa = batch['fa']
        fa = tf.where(tf.math.is_nan(fa), tf.zeros_like(fa), fa)
        fa = (fa-fa_mean)/fa_std
        md = batch['md']
        md = tf.where(tf.math.is_nan(md), tf.zeros_like(md), md)
        md = (md-md_mean)/md_std
        rd = batch['rd']
        rd = tf.where(tf.math.is_nan(rd), tf.zeros_like(rd), rd)
        rd = (rd-rd_mean)/rd_std
    subjectid = decoder(batch['subjectid'])
    y = get_labels(df, subjectid, list(cat_cols.keys())+num_cols)
    #X = tf.concat([t1], axis=4)
    X = tf.concat([t1, t2], axis=4)
    if optimizer != None:
      y_true, y_pred, loss = train_step(X, y, model, optimizer, cat_cols, num_cols)
    else:
      y_true, y_pred = test_step(X, y, model)
    out_loss, out_acc = calc_loss_acc(out_loss, out_acc, y_true, y_pred, cat_cols, num_cols, norm_dict)
    n += X.shape[0]
    n_batch += 1
    if (n_batch % 10) == 0:
      print(n_batch)
    total_time_model += time.time() - start_time
    start_time = time.time()
  return (out_loss, out_acc, n, total_time_model, total_time_dataload)

def get_labels(df, subjectid, cols = ['nihtbx_fluidcomp_uncorrected_norm']):
  subjects_df = pd.DataFrame(subjectid)
  result_df = pd.merge(subjects_df, df, left_on=0, right_on='subjectkey', how='left')
  output = {}
  for col in cols:
    output[col] = np.asarray(result_df[col].values)
  return output

def best_val(df_best, df_val, df_test, e):
  df_best = pd.merge(df_best, df_val, how='left', left_on='name', right_on='name')
  df_best = pd.merge(df_best, df_test, how='left', left_on='name', right_on='name')
  df_best.loc[df_best['best_loss_val']>=df_best['cur_loss_val'], 'best_loss_epochs'] = e
  df_best.loc[(df_best['best_acc_val']<=df_best['cur_acc_val'])&(df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'best_acc_epochs'] = e
  df_best.loc[df_best['best_loss_val']>=df_best['cur_loss_val'], 'best_loss_test'] = df_best.loc[df_best['best_loss_val']>=df_best['cur_loss_val'], 'cur_loss_test']
  df_best.loc[df_best['best_loss_val']>=df_best['cur_loss_val'], 'best_loss_val'] = df_best.loc[df_best['best_loss_val']>=df_best['cur_loss_val'], 'cur_loss_val']
  df_best.loc[(df_best['best_acc_val']<=df_best['cur_acc_val'])&(df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'best_acc_test'] = df_best.loc[(df_best['best_acc_val']<=df_best['cur_acc_val'])&(df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'cur_acc_test']
  df_best.loc[(df_best['best_acc_val']<=df_best['cur_acc_val'])&(df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'best_acc_val'] = df_best.loc[(df_best['best_acc_val']<=df_best['cur_acc_val'])&(df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'cur_acc_val']
  df_best.loc[(df_best['best_acc_val']>=df_best['cur_acc_val'])&(~df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'best_acc_test'] = df_best.loc[(df_best['best_acc_val']>=df_best['cur_acc_val'])&(~df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'cur_acc_test']
  df_best.loc[(df_best['best_acc_val']>=df_best['cur_acc_val'])&(~df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'best_acc_val'] = df_best.loc[(df_best['best_acc_val']>=df_best['cur_acc_val'])&(~df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'cur_acc_val']
  df_best = df_best.drop(['cur_loss_val', 'cur_acc_val', 'cur_loss_test', 'cur_acc_test'], axis=1)
  return(df_best)

In [ ]:
decoder = np.vectorize(lambda x: x.decode('UTF-8'))
template = 'Epoch {0}, Loss: {1:.3f}, Accuracy: {2:.3f}, Val Loss: {3:.3f}, Val Accuracy: {4:.3f}, Time Model: {5:.3f}, Time Data: {6:.3f}'
for col in [0]:
  log_textfile(path_output + modelname + '/log' + '.log', cat_cols),
  log_textfile(path_output + modelname + '/log' + '.log', num_cols)
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
  optimizer = tf.keras.optimizers.Adam(lr = 0.001)
  model = Model(cat_cols, num_cols)
  df_best = None
  for e in range(20):
    log_textfile(path_output + modelname + '/log' + '.log', 'Epochs: ' + str(e))
    loss = tf.Variable(0.)
    acc = tf.Variable(0.) 
    val_loss = tf.Variable(0.)
    val_acc = tf.Variable(0.)
    test_loss = tf.Variable(0.)
    test_acc = tf.Variable(0.)
    tf.keras.backend.set_learning_phase(True)
    train_out_loss, train_out_acc, n, time_model, time_data = epoch(train_iter, train_df, model, optimizer, cat_cols, num_cols, norm_dict)
    tf.keras.backend.set_learning_phase(False)
    val_out_loss, val_out_acc, n, _, _ = epoch(val_iter, val_df, model, None, cat_cols, num_cols, norm_dict)
    test_out_loss, test_out_acc, n, _, _ = epoch(test_iter, test_df, model, None, cat_cols, num_cols, norm_dict)
    loss, acc, _ = format_output(train_out_loss, train_out_acc, n, list(cat_cols.keys())+num_cols)
    val_loss, val_acc, df_val = format_output(val_out_loss, val_out_acc, n, list(cat_cols.keys())+num_cols, print_bl=False)
    test_loss, test_acc, df_test = format_output(test_out_loss, test_out_acc, n, list(cat_cols.keys())+num_cols, print_bl=False)
    df_val.columns = ['name', 'cur_loss_val', 'cur_acc_val']
    df_test.columns = ['name', 'cur_loss_test', 'cur_acc_test']
    if e == 0:
      df_best = pd.merge(df_test, df_val, how='left', left_on='name', right_on='name')
      df_best['best_acc_epochs'] = 0
      df_best['best_loss_epochs'] = 0
      df_best.columns = ['name', 'best_loss_test', 'best_acc_test', 'best_loss_val', 'best_acc_val', 'best_acc_epochs', 'best_loss_epochs']
    df_best = best_val(df_best, df_val, df_test, e)
    print(df_best[['name', 'best_loss_test', 'best_acc_test']])
    print(df_best[['name', 'best_loss_val', 'best_acc_val']])
    log_textfile(path_output + modelname + '/log' + '.log', template.format(e, loss, acc, val_loss, val_acc, time_model, time_data))
    if e in [10, 15]:
      optimizer.lr = optimizer.lr/3
      log_textfile(path_output + modelname + '/log' + '.log', 'Learning rate: ' + str(optimizer.lr))
    df_best.to_csv(path_output +  modelname + '/df_best' + str(e) + '.csv')
    df_best.to_csv(path_output +  modelname + '/df_best' + '.csv')
    model.save_weights(path_output + modelname + '/checkpoints/' + str(e) + '/')

In [ ]:
error

In [ ]:
test_loss, test_acc, df_test = format_output(test_out_loss, test_out_acc, n, list(cat_cols.keys())+num_cols, print_bl=False)

In [ ]:
df_test.to_csv('final_output_all.csv')

In [ ]:
inputs = tf.keras.Input(shape=(64,64,64,2), name='inputlayer123')
a = model(inputs)['female']
mm = tf.keras.models.Model(inputs=inputs, outputs=a)

In [ ]:
from tf_explain.core.smoothgrad import SmoothGrad
import pickle

explainer = SmoothGrad()
output_grid = {}
output_n = {}
for i in range(2):
  output_grid[i] = np.zeros((64,64,64))
  output_n[i] = 0
counter = 0
for batch in test_iter:
  counter+=1
  print(counter)
  t1 = (tf.cast(batch['t1'], tf.float32)-t1_mean)/t1_std
  t2 = (batch['t2']-t2_mean)/t2_std
  X = tf.concat([t1, t2], axis=4)
  subjectid = decoder(batch['subjectid'])
  y = get_labels(test_df, subjectid, list(cat_cols.keys())+num_cols)
  y_list = list(y['female'])
  for i in range(X.shape[0]):
    X_i = X[i]
    X_i = tf.expand_dims(X_i, axis=0)
    y_i = y_list[i]
    grid = explainer.explain((X_i, _), mm, y_i, 20, 1.)
    output_grid[y_i] += grid
    output_n[y_i] += 1

In [ ]:
pickle.dump([output_grid, output_n], open( "smoothgrad_female_all.p", "wb" ) )

In [ ]:
#output_grid, output_n = pickle.load(open( "smoothgrad_female.p", "rb" ))

In [ ]:
def apply_grey_patch(image, top_left_x, top_left_y, top_left_z, patch_size):
    """
    Replace a part of the image with a grey patch.
    Args:
        image (numpy.ndarray): Input image
        top_left_x (int): Top Left X position of the applied box
        top_left_y (int): Top Left Y position of the applied box
        patch_size (int): Size of patch to apply
    Returns:
        numpy.ndarray: Patched image
    """
    patched_image = np.array(image, copy=True)
    patched_image[
        top_left_x : top_left_x + patch_size, top_left_y : top_left_y + patch_size, top_left_z : top_left_z + patch_size, :
    ] = 0

    return patched_image

In [ ]:
import math

In [ ]:
def get_sensgrid(image, mm, class_index, patch_size):
  sensitivity_map = np.zeros((
    math.ceil(image.shape[0] / patch_size),
    math.ceil(image.shape[1] / patch_size),
    math.ceil(image.shape[2] / patch_size)
  ))
  for index_z, top_left_z in enumerate(range(0, image.shape[2], patch_size)):
    patches = [
              apply_grey_patch(image, top_left_x, top_left_y, top_left_z, patch_size)
              for index_x, top_left_x in enumerate(range(0, image.shape[0], patch_size))
              for index_y, top_left_y in enumerate(range(0, image.shape[1], patch_size))
              ]
    coordinates = [
                (index_y, index_x)
                for index_x, _ in enumerate(range(0, image.shape[0], patch_size))
                for index_y, _ in enumerate(range(0, image.shape[1], patch_size))
                ]
    predictions = mm.predict(np.array(patches), batch_size=1)
    target_class_predictions = [prediction[class_index] for prediction in predictions]
    for (index_y, index_x), confidence in zip(coordinates, target_class_predictions):
      sensitivity_map[index_y, index_x, index_z] = 1 - confidence
  sm = resize(sensitivity_map, (64,64,64))
  heatmap = (sm - np.min(sm)) / (sm.max() - sm.min())
  return(heatmap)

In [ ]:
output_grid = {}
output_n = {}
for i in range(2):
  output_grid[i] = np.zeros((64,64,64))
  output_n[i] = 0

counter = 0
for batch in test_iter:
  counter+=1
  print(counter)
  t1 = (tf.cast(batch['t1'], tf.float32)-t1_mean)/t1_std
  t2 = (batch['t2']-t2_mean)/t2_std
  X = tf.concat([t1, t2], axis=4)
  subjectid = decoder(batch['subjectid'])
  y = get_labels(test_df, subjectid, list(cat_cols.keys())+num_cols)
  y_list = list(y['female'])
  for i in range(X.shape[0]):
    print(i)
    X_i = X[i]
    y_i = y_list[i]
    grid = get_sensgrid(X_i, mm, y_i, 4)
    output_grid[y_i] += grid
    output_n[y_i] += 1
  if counter==6:
    break

In [ ]:
pickle.dump([output_grid, output_n], open( "heatmap_female_all.p", "wb" ) )

In [ ]:
error

In [ ]:
batch = next(iter(train_iter))

In [ ]:
t1 = (tf.cast(batch['t1'], tf.float32)-t1_mean)/t1_std
t2 = (batch['t2']-t2_mean)/t2_std
ad = batch['ad']
ad = tf.where(tf.math.is_nan(ad), tf.zeros_like(ad), ad)
ad = (ad-ad_mean)/ad_std
fa = batch['fa']
fa = tf.where(tf.math.is_nan(fa), tf.zeros_like(fa), fa)
fa = (fa-fa_mean)/fa_std
md = batch['md']
md = tf.where(tf.math.is_nan(md), tf.zeros_like(md), md)
md = (md-md_mean)/md_std
rd = batch['rd']
rd = tf.where(tf.math.is_nan(rd), tf.zeros_like(rd), rd)
rd = (rd-rd_mean)/rd_std
#subjectid = decoder(batch['subjectid'])
#y = get_labels(df, subjectid, list(cat_cols.keys())+num_cols)
#X = tf.concat([t1, t2, ad, fa, md, rd], axis=4)
X = tf.concat([t1, t2], axis=4)    

In [ ]:
tf.keras.backend.set_learning_phase(True)
model(X)['female']

In [ ]:
tf.keras.backend.set_learning_phase(False)
model(X)['female']

In [ ]:
mean_std = [x.name for x in model.non_trainable_variables if ('batch_norm') in x.name and ('mean' in x.name or 'variance' in x.name)]

In [ ]:
model = Model(cat_cols, num_cols)

In [ ]:
model.non_trainable_variables 